# Loading the models 
I used the provided libraries from https://github.com/princeton-nlp/SimCSE 

There are different models can be used for sentence embeddings with SimCSE such as "bert-based-uncased", "Robertta-based" or "Roberta-larged".
Here, we chose the "princeton-nlp/sup-simcse-roberta-large"

In [ ]:
import numpy as np
import csv
import pandas as pd
from simcse import SimCSE
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

device = "cuda:0" if torch.cuda.is_available() else "cpu"


tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")

model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large").to(device)

print('Model is loaded')

# READ the DATA

In [ ]:
Test_data = pd.read_csv("/home/ubuntu/cluster/Test_CLEAN2.csv", error_bad_lines=False, delimiter= '\t')
print('Data is loaded')

In [ ]:
# Prepare the data. We onlye need to extrat the features for the conversations, so we drop the other columns!

data = Test_data.drop(['user1_MSG'], axis=1)
data = data.drop(['user2_MSG'], axis=1)
data = data.drop(['User1_ID'], axis=1)
data = data.drop(['User2_ID'], axis=1)



main_msg = data['conversations']
print('Data saving is ready')

# compute and save the features to a file

In [ ]:
with open('/home/ubuntu/cluster/Test_Embeddings_roberta-large.csv', 'w', newline='') as f:
      

#len(main_msg)



    for i in range(len(main_msg)):
        print(i)
        inputs = tokenizer(main_msg[i], padding=True, truncation=True, return_tensors="pt").to(device)

    

        with torch.no_grad():

            embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
            writer = csv.writer(f, delimiter=',')

            writer.writerows(embeddings.cpu().numpy())

f.close()

#*********************

#Merge the features with their labels

features = pd.read_csv('/home/ubuntu/cluster/Test_Embeddings_roberta-large.csv', header=None,delimiter= ',')

data.drop("conversations", axis = 1, inplace = True)

result_features= pd.concat([data,features], axis=1)



        

#save results in a file



result_features.to_csv("/home/ubuntu/cluster/SIMCe_TEST_Embeddings_roberta-large.csv")